# Data Cleaning and Wrangling

# Let's check the Data for Inconsistencies

In [1]:
#Let's Check Data Consistency and see if there are any Missing Values or Inconsistencies

# Importing necessary libraries

In [2]:
import pandas as pd

import requests

from datetime import datetime, timedelta, timezone
import time
import ntplib

import sys
sys.path.append("../scripts/")
import path

# Getting Curent Date

In [3]:
#.replace(hour=0, minute=0, second=0) <- Used to make the Time from yesterday at 23, to gather all Data until Yesterday INCLUDED.

#On Server with working clock
#yesterday = (datetime.now() - timedelta(days=1)).replace(hour=0, minute=0, second=0).strftime("%Y-%m-%d %H:%M:%S")

#Getting Actual Data from the Internet for Server with messy clock
try:
    client = ntplib.NTPClient()
    response = client.request('pool.ntp.org')
    yesterday = (datetime.fromtimestamp(response.tx_time, tz=timezone.utc) - timedelta(days=1)).replace(hour=23, minute=0, second=0).strftime("%Y-%m-%d %H:%M:%S")
    print(yesterday)

except:
    print("Could not sync with time server.")

2024-01-10 23:00:00


In [4]:
startdate = (datetime.strptime(yesterday, "%Y-%m-%d %H:%M:%S") - timedelta(weeks=108)).replace(hour=0, minute=0, second=0)
enddate = datetime.strptime(yesterday, "%Y-%m-%d %H:%M:%S").replace(hour=23, minute=0, second=0)

# Importing Raw Data

In [5]:
RawData = pd.read_parquet(path.RAW_DATA_DIR / "BTC-USD_HourlyRawData_From2021-12-14 00:00:00_To2024-01-09 23:00:00.parquet")

RawData

,Date,Open,High,Low,Close,Volume
298,2021-12-14 00:00:00+00:00,46300.00,47243.18,46727.89,47022.75,870.201420
297,2021-12-14 01:00:00+00:00,46607.90,47128.52,47018.78,46889.47,559.425978
296,2021-12-14 02:00:00+00:00,46700.26,47081.39,46899.17,47052.39,466.226036
295,2021-12-14 03:00:00+00:00,46782.86,47130.37,47052.39,46977.81,328.500181
294,2021-12-14 04:00:00+00:00,46890.12,47207.92,46977.80,47017.01,519.013109
...,...,...,...,...,...,...
17940,2024-01-09 19:00:00+00:00,46589.25,46903.46,46658.71,46890.10,653.207706
17939,2024-01-09 20:00:00+00:00,46593.78,46940.00,46888.27,46651.31,1086.527743
17938,2024-01-09 21:00:00+00:00,44701.54,47901.00,46649.39,45419.45,8919.956620
17937,2024-01-09 22:00:00+00:00,45275.03,46286.39,45420.10,46278.06,1915.366592


In [6]:
#Cloning the DF for transformations

TransformedData = RawData.copy()

# The WebAPI isn't consistent, as also described in the docs, we'll need to make a couple of checks.

In [7]:
#Docs at https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles

# Checking if the First Value and Last Value are at the Start and End Dates

In [8]:
print(startdate.replace(tzinfo=timezone.utc) == TransformedData["Date"].iloc[0])
print(enddate.replace(tzinfo=timezone.utc) == TransformedData["Date"].iloc[-1])

False
False


# Checking for Duplicates

In [9]:
Duplicates = TransformedData.loc[TransformedData.duplicated() == True]
Duplicates

print(Duplicates["Date"])

0       2021-12-26 10:00:00+00:00
299     2022-01-07 20:00:00+00:00
1195    2022-01-20 06:00:00+00:00
1494    2022-02-01 16:00:00+00:00
1793    2022-02-14 02:00:00+00:00
1495    2022-02-26 12:00:00+00:00
1794    2022-03-10 22:00:00+00:00
2690    2022-03-23 08:00:00+00:00
2392    2022-04-04 18:00:00+00:00
3288    2022-04-17 04:00:00+00:00
2990    2022-04-29 14:00:00+00:00
3289    2022-05-12 00:00:00+00:00
4185    2022-05-24 10:00:00+00:00
4484    2022-06-05 20:00:00+00:00
4783    2022-06-18 06:00:00+00:00
5082    2022-06-30 16:00:00+00:00
5381    2022-07-13 02:00:00+00:00
5680    2022-07-25 12:00:00+00:00
5979    2022-08-06 22:00:00+00:00
6278    2022-08-19 08:00:00+00:00
5980    2022-08-31 18:00:00+00:00
6876    2022-09-13 04:00:00+00:00
7175    2022-09-25 14:00:00+00:00
7474    2022-10-08 00:00:00+00:00
7773    2022-10-20 10:00:00+00:00
7475    2022-11-01 20:00:00+00:00
7774    2022-11-14 06:00:00+00:00
8670    2022-11-26 16:00:00+00:00
8969    2022-12-09 02:00:00+00:00
9268    2022-1

In [10]:
print(f'At Net of Duplicates the Len of the Records is: {len(RawData) - len(RawData.loc[RawData.duplicated() == True])}')

At Net of Duplicates the Len of the Records is: 18164


In [11]:
#Removing Duplicates

TransformedData.drop_duplicates(keep="first", inplace = True)
TransformedData

,Date,Open,High,Low,Close,Volume
298,2021-12-14 00:00:00+00:00,46300.00,47243.18,46727.89,47022.75,870.201420
297,2021-12-14 01:00:00+00:00,46607.90,47128.52,47018.78,46889.47,559.425978
296,2021-12-14 02:00:00+00:00,46700.26,47081.39,46899.17,47052.39,466.226036
295,2021-12-14 03:00:00+00:00,46782.86,47130.37,47052.39,46977.81,328.500181
294,2021-12-14 04:00:00+00:00,46890.12,47207.92,46977.80,47017.01,519.013109
...,...,...,...,...,...,...
17940,2024-01-09 19:00:00+00:00,46589.25,46903.46,46658.71,46890.10,653.207706
17939,2024-01-09 20:00:00+00:00,46593.78,46940.00,46888.27,46651.31,1086.527743
17938,2024-01-09 21:00:00+00:00,44701.54,47901.00,46649.39,45419.45,8919.956620
17937,2024-01-09 22:00:00+00:00,45275.03,46286.39,45420.10,46278.06,1915.366592


# Checking for Missing Values

In [12]:
diff = enddate - startdate

print(f'Total Hours Elapsed (Expected Num of Candles): {diff.total_seconds() //3600 + 1}') # +1 as the last Candle is Included

Total Hours Elapsed (Expected Num of Candles): 18168.0


In [13]:
print(f'Number of Missing Candles: {(diff.total_seconds() //3600) + 1 - len(TransformedData)}') # +1 as the last Candle is Included

Number of Missing Candles: 4.0


# Retrieving Missing Data Dates and Filling it with with a Doji Candle (Same Close as Previous Candle and same Open High Low Close Prices)

In [14]:
Dates = list(TransformedData["Date"])
Missing = []

testdate = Dates[0]

while testdate <= Dates[-1]:
    if testdate not in Dates:
        Missing.append(testdate)
        
    testdate += timedelta(hours=1)

In [15]:
Missing

[Timestamp('2023-03-04 17:00:00+0000', tz='UTC'),
 Timestamp('2023-03-04 18:00:00+0000', tz='UTC'),
 Timestamp('2023-03-04 19:00:00+0000', tz='UTC'),
 Timestamp('2023-03-04 20:00:00+0000', tz='UTC')]

In [16]:
RecordToCopy = TransformedData[TransformedData["Date"] == Missing[0] - timedelta(hours=1)]

In [17]:
templist = [[x, RecordToCopy.iloc[0]["Close"], RecordToCopy.iloc[0]["Close"], RecordToCopy.iloc[0]["Close"], RecordToCopy.iloc[0]["Close"],RecordToCopy.iloc[0]["Volume"]] for x in Missing]
tempdf = pd.DataFrame(templist, columns=["Date", "Open", "High", "Low", "Close", "Volume"])

TransformedData = pd.concat([TransformedData, tempdf])

In [18]:
TransformedData

,Date,Open,High,Low,Close,Volume
298,2021-12-14 00:00:00+00:00,46300.00,47243.18,46727.89,47022.75,870.201420
297,2021-12-14 01:00:00+00:00,46607.90,47128.52,47018.78,46889.47,559.425978
296,2021-12-14 02:00:00+00:00,46700.26,47081.39,46899.17,47052.39,466.226036
295,2021-12-14 03:00:00+00:00,46782.86,47130.37,47052.39,46977.81,328.500181
294,2021-12-14 04:00:00+00:00,46890.12,47207.92,46977.80,47017.01,519.013109
...,...,...,...,...,...,...
17936,2024-01-09 23:00:00+00:00,45933.43,46276.18,46272.94,46124.08,913.480062
0,2023-03-04 17:00:00+00:00,22330.27,22330.27,22330.27,22330.27,31.413194
1,2023-03-04 18:00:00+00:00,22330.27,22330.27,22330.27,22330.27,31.413194
2,2023-03-04 19:00:00+00:00,22330.27,22330.27,22330.27,22330.27,31.413194


In [19]:
#Sort Again for new Values
TransformedData = TransformedData.sort_values(by="Date")
TransformedData

,Date,Open,High,Low,Close,Volume
298,2021-12-14 00:00:00+00:00,46300.00,47243.18,46727.89,47022.75,870.201420
297,2021-12-14 01:00:00+00:00,46607.90,47128.52,47018.78,46889.47,559.425978
296,2021-12-14 02:00:00+00:00,46700.26,47081.39,46899.17,47052.39,466.226036
295,2021-12-14 03:00:00+00:00,46782.86,47130.37,47052.39,46977.81,328.500181
294,2021-12-14 04:00:00+00:00,46890.12,47207.92,46977.80,47017.01,519.013109
...,...,...,...,...,...,...
17940,2024-01-09 19:00:00+00:00,46589.25,46903.46,46658.71,46890.10,653.207706
17939,2024-01-09 20:00:00+00:00,46593.78,46940.00,46888.27,46651.31,1086.527743
17938,2024-01-09 21:00:00+00:00,44701.54,47901.00,46649.39,45419.45,8919.956620
17937,2024-01-09 22:00:00+00:00,45275.03,46286.39,45420.10,46278.06,1915.366592


# Dropping Useless Columns

In [20]:
#For the sake of simplicity in Model Design, we'll just use Closing Prices and treat it as a Pure Time-Series

In [21]:
TransformedData.drop(["Open", "High", "Low", "Volume"], axis=1, inplace=True)

In [22]:
TransformedData

,Date,Close
298,2021-12-14 00:00:00+00:00,47022.75
297,2021-12-14 01:00:00+00:00,46889.47
296,2021-12-14 02:00:00+00:00,47052.39
295,2021-12-14 03:00:00+00:00,46977.81
294,2021-12-14 04:00:00+00:00,47017.01
...,...,...
17940,2024-01-09 19:00:00+00:00,46890.10
17939,2024-01-09 20:00:00+00:00,46651.31
17938,2024-01-09 21:00:00+00:00,45419.45
17937,2024-01-09 22:00:00+00:00,46278.06


# Dumping Cleaned Data to Disk

In [30]:
start = (TransformedData["Date"].iloc[0])
finish = (TransformedData["Date"].iloc[-1])
product_id = "BTC-USD"

TransformedData.to_parquet(path.TRANSFORMED_DATA_DIR / f'{product_id}_HourlyTransformedData_From{start}_To{finish}.parquet')